## Preprocessing

In [1]:
# Import our dependencies
from google.colab import files
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import io

#  Import and read the charity_data.csv.
uploaded = files.upload()
application_df = pd.read_csv(io.StringIO(uploaded['charity_data.csv'].decode('utf-8')))
application_df.head()

Saving charity_data.csv to charity_data.csv


,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df = application_df.drop(['EIN','NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column.
unique = application_df.nunique()
unique

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning
application_types_counts = application_df['APPLICATION_TYPE'].value_counts()
application_types_counts

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Remove T12,T2,T25,T14,T29,T15,T17
application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].loc[~((application_df['APPLICATION_TYPE']== 'T12')|
                                                                              (application_df['APPLICATION_TYPE']== 'T2')|
                                                                              (application_df['APPLICATION_TYPE']== 'T25')|
                                                                              (application_df['APPLICATION_TYPE']== 'T14')|
                                                                              (application_df['APPLICATION_TYPE']== 'T29')|
                                                                              (application_df['APPLICATION_TYPE']== 'T15')|
                                                                              (application_df['APPLICATION_TYPE']== 'T17'))]

# # Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
Name: APPLICATION_TYPE, dtype: int64

In [6]:
# Look at CLASSIFICATION value counts for binning
CLASSIFICATION_counts = application_df['CLASSIFICATION'].value_counts()
CLASSIFICATION_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts>1
CLASSIFICATION_counts = application_df['CLASSIFICATION'].value_counts().loc[lambda x:x>1]
CLASSIFICATION_counts

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [8]:
# Remove CLASSIFICATION value counts<10
CLASSIFICATION_remove_list = application_df['CLASSIFICATION'].value_counts().loc[lambda x:x<20].index.tolist()

for i in CLASSIFICATION_remove_list:
  application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].loc[~(application_df['CLASSIFICATION']==i)]
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
Name: CLASSIFICATION, dtype: int64

In [9]:
application_df['STATUS'].value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [10]:
# Drop 'STATUS'
application_df = application_df.loc[application_df['STATUS']==1].drop('STATUS',axis=1)

In [11]:
application_df['SPECIAL_CONSIDERATIONS'].value_counts()

N    34268
Y       26
Name: SPECIAL_CONSIDERATIONS, dtype: int64

In [13]:
application_df= application_df.reset_index(drop=True)

In [14]:
application_df['INCOME_AMT'].value_counts()

0                24385
25000-99999       3746
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        542
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

In [15]:
# INCOME_AMT has difference in grades
for i in range (0, len(application_df)):
  if application_df.loc[i,'INCOME_AMT']==0:
    application_df.loc[i,'INCOME_AMT']= 1
  elif application_df.loc[i,'INCOME_AMT']=='1-9999':
    application_df.loc[i,'INCOME_AMT']= 2
  elif application_df.loc[i,'INCOME_AMT']=='10000-24999':
    application_df.loc[i,'INCOME_AMT']= 3
  elif application_df.loc[i,'INCOME_AMT']=='25000-99999':
    application_df.loc[i,'INCOME_AMT']= 4
  elif application_df.loc[i,'INCOME_AMT']=='100000-499999':
    application_df.loc[i,'INCOME_AMT']= 5    
  elif application_df.loc[i,'INCOME_AMT']=='1M-5M':
    application_df.loc[i,'INCOME_AMT']= 6
  elif application_df.loc[i,'INCOME_AMT']=='5M-10M':
    application_df.loc[i,'INCOME_AMT']= 7
  elif application_df.loc[i,'INCOME_AMT']=='10M-50M':
    application_df.loc[i,'INCOME_AMT']= 8
  elif application_df.loc[i,'INCOME_AMT']=='50M+':
    application_df.loc[i,'INCOME_AMT']= 9

#Transform string to int
application_df['INCOME_AMT']=application_df['INCOME_AMT'].astype('int32')

# Check to make sure binning was successful
application_df['INCOME_AMT'].value_counts()

0    24385
4     3746
5     3374
6      955
2      728
3      542
8      240
7      185
9      139
Name: INCOME_AMT, dtype: int64

In [16]:
application_df['ASK_AMT'].value_counts()

5000        25394
63981           3
15583           3
6725            3
10478           3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [17]:
# Bin ASK_AMT value based on 'INCOME_AMT'

for i in range (0, len(application_df)):
  if (application_df.loc[i,'ASK_AMT']>=1)&(application_df.loc[i,'ASK_AMT']<=9999):
    application_df.loc[i,'ASK_AMT']=1
  elif (application_df.loc[i,'ASK_AMT']>=10000)&(application_df.loc[i,'ASK_AMT']<=24999):
    application_df.loc[i,'ASK_AMT']=2
  elif (application_df.loc[i,'ASK_AMT']>=25000)&(application_df.loc[i,'ASK_AMT']<=99999):
    application_df.loc[i,'ASK_AMT']=3 
  elif (application_df.loc[i,'ASK_AMT']>=100000)&(application_df.loc[i,'ASK_AMT']<=499999):
    application_df.loc[i,'ASK_AMT']=4
  elif (application_df.loc[i,'ASK_AMT']>=500000)&(application_df.loc[i,'ASK_AMT']<=999999):
    application_df.loc[i,'ASK_AMT']=5    
  elif (application_df.loc[i,'ASK_AMT']>=1000000)&(application_df.loc[i,'ASK_AMT']<=4999999):
    application_df.loc[i,'ASK_AMT']=6
  elif (application_df.loc[i,'ASK_AMT']>=5000000)&(application_df.loc[i,'ASK_AMT']<=9999999):
    application_df.loc[i,'ASK_AMT']=7
  elif (application_df.loc[i,'ASK_AMT']>=10000000)&(application_df.loc[i,'ASK_AMT']<=49999999):
    application_df.loc[i,'ASK_AMT']=8
  elif (application_df.loc[i,'ASK_AMT']>=50000000):
    application_df.loc[i,'ASK_AMT']=9


# Check to make sure binning was successful
application_df['ASK_AMT'].value_counts()

1    25941
3     2685
4     2305
2     1135
6      940
5      651
8      258
7      225
9      154
Name: ASK_AMT, dtype: int64

In [18]:
# Convert categorical data to numeric with `pd.get_dummies`
dummied = pd.get_dummies(application_df)
dummied 

,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_T10,APPLICATION_TYPE_T13,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,USE_CASE_Heathcare,USE_CASE_Other,USE_CASE_Preservation,USE_CASE_ProductDev,ORGANIZATION_Association,ORGANIZATION_Co-operative,ORGANIZATION_Corporation,ORGANIZATION_Trust,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,0,1,1,1,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,1,0
1,2,4,1,0,0,0,1,0,0,0,...,0,0,1,0,0,1,0,0,1,0
2,0,1,0,0,0,0,0,0,1,0,...,0,0,0,1,1,0,0,0,1,0
3,3,1,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,1,1,0
4,5,4,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34289,0,1,0,0,0,0,0,1,0,0,...,0,0,0,1,1,0,0,0,1,0
34290,0,1,0,0,0,0,0,1,0,0,...,0,0,0,1,1,0,0,0,1,0
34291,0,1,0,0,0,0,1,0,0,0,...,0,0,1,0,1,0,0,0,1,0
34292,0,1,1,0,0,0,0,0,1,0,...,0,0,0,1,1,0,0,0,1,0


In [19]:
# Split data into features and target arrays
X = dummied.drop('IS_SUCCESSFUL',axis=1)
y = dummied['IS_SUCCESSFUL']
y.value_counts()

1    18258
0    16036
Name: IS_SUCCESSFUL, dtype: int64

In [20]:
# Split the preprocessed data into a training and testing dataset
X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=23,stratify=y, test_size=0.2)

In [21]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [29]:
# For-loop to get the best model
testsets=[]
for i in [128,256]:
  for j in [16,32,64]:
    for k in [16,32]:
      for l in [4,8]:
        nn_test = tf.keras.models.Sequential()
        # Input layer
        nn_test.add(tf.keras.layers.Dense(units = i, activation = 'relu', input_dim = 50))
        # First hidden layer
        nn_test.add(tf.keras.layers.Dense(units = j, activation = 'relu'))
        # Second hidden layer
        nn_test.add(tf.keras.layers.Dense(units = k, activation = 'relu'))
        # Third hidden layer
        nn_test.add(tf.keras.layers.Dense(units = l, activation = 'relu'))
        # Output layer
        nn_test.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

        # Compile the model
        nn_test.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])
      
        for u in [20,30,40]:
          fit_model = nn_test.fit(X_train_scaled,y_train,epochs =u)
          model_loss, model_accuracy = nn_test.evaluate(X_test_scaled,y_test,verbose=2)
          testset = {'match':[i,j,k,l,u],'Loss':model_loss,'Accuracy':model_accuracy}
          testsets.append(testset)

Epoch 1/20
858/858 [==============================] - 4s 2ms/step - loss: 0.6072 - accuracy: 0.7064
Epoch 2/20
858/858 [==============================] - 2s 2ms/step - loss: 0.5563 - accuracy: 0.7257
Epoch 3/20
858/858 [==============================] - 3s 4ms/step - loss: 0.5507 - accuracy: 0.7304
Epoch 4/20
858/858 [==============================] - 5s 6ms/step - loss: 0.5493 - accuracy: 0.7303
Epoch 5/20
858/858 [==============================] - 2s 2ms/step - loss: 0.5479 - accuracy: 0.7308
Epoch 6/20
858/858 [==============================] - 2s 2ms/step - loss: 0.5468 - accuracy: 0.7320
Epoch 7/20
858/858 [==============================] - 2s 2ms/step - loss: 0.5462 - accuracy: 0.7315
Epoch 8/20
858/858 [==============================] - 3s 3ms/step - loss: 0.5462 - accuracy: 0.7326
Epoch 9/20
858/858 [==============================] - 3s 4ms/step - loss: 0.5453 - accuracy: 0.7321
Epoch 10/20
858/858 [==============================] - 3s 3ms/step - loss: 0.5446 - accuracy: 0.7348

In [30]:
print(testsets)

[{'match': [128, 16, 16, 4, 20], 'Loss': 0.5463634133338928, 'Accuracy': 0.7302813529968262}, {'match': [128, 16, 16, 4, 30], 'Loss': 0.5459840297698975, 'Accuracy': 0.7314477562904358}, {'match': [128, 16, 16, 4, 40], 'Loss': 0.5494163036346436, 'Accuracy': 0.7296981811523438}, {'match': [128, 16, 16, 8, 20], 'Loss': 0.5496904850006104, 'Accuracy': 0.7343636155128479}, {'match': [128, 16, 16, 8, 30], 'Loss': 0.5442466139793396, 'Accuracy': 0.7313019633293152}, {'match': [128, 16, 16, 8, 40], 'Loss': 0.5489598512649536, 'Accuracy': 0.7294065952301025}, {'match': [128, 16, 32, 4, 20], 'Loss': 0.5465701222419739, 'Accuracy': 0.7261991500854492}, {'match': [128, 16, 32, 4, 30], 'Loss': 0.5469508767127991, 'Accuracy': 0.7291150093078613}, {'match': [128, 16, 32, 4, 40], 'Loss': 0.5468907356262207, 'Accuracy': 0.7340720295906067}, {'match': [128, 16, 32, 8, 20], 'Loss': 0.5442301630973816, 'Accuracy': 0.7331972718238831}, {'match': [128, 16, 32, 8, 30], 'Loss': 0.5473054051399231, 'Accuracy

In [36]:
df = pd.DataFrame(testsets)
df

,match,Loss,Accuracy
0,"[128, 16, 16, 4, 20]",0.546363,0.730281
1,"[128, 16, 16, 4, 30]",0.545984,0.731448
2,"[128, 16, 16, 4, 40]",0.549416,0.729698
3,"[128, 16, 16, 8, 20]",0.549690,0.734364
4,"[128, 16, 16, 8, 30]",0.544247,0.731302
...,...,...,...
67,"[256, 64, 32, 4, 30]",0.552766,0.729698
68,"[256, 64, 32, 4, 40]",0.553333,0.731739
69,"[256, 64, 32, 8, 20]",0.544805,0.733051
70,"[256, 64, 32, 8, 30]",0.549150,0.730281


In [38]:
df.sort_values('Accuracy',ascending=False).head(1)

,match,Loss,Accuracy
51,"[256, 32, 16, 8, 20]",0.544602,0.735093


In [40]:
# Create a good model

nn = tf.keras.models.Sequential()

# Input layer
nn.add(tf.keras.layers.Dense(units = 256, activation = 'relu', input_dim = 50))
# First hidden layer
nn.add(tf.keras.layers.Dense(units = 32, activation = 'relu'))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units = 16, activation = 'relu'))
# Third hidden layer
nn.add(tf.keras.layers.Dense(units = 8, activation = 'relu'))
# Output layer
nn.add(tf.keras.layers.Dense(units = 1, activation = 'sigmoid'))

# Compile the model
nn.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])

In [41]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs =20)

Epoch 1/20
858/858 [==============================] - 3s 3ms/step - loss: 0.6059 - accuracy: 0.7198
Epoch 2/20
858/858 [==============================] - 2s 3ms/step - loss: 0.5789 - accuracy: 0.7274
Epoch 3/20
858/858 [==============================] - 2s 3ms/step - loss: 0.5683 - accuracy: 0.7264
Epoch 4/20
858/858 [==============================] - 2s 3ms/step - loss: 0.5627 - accuracy: 0.7292
Epoch 5/20
858/858 [==============================] - 2s 3ms/step - loss: 0.5594 - accuracy: 0.7294
Epoch 6/20
858/858 [==============================] - 2s 3ms/step - loss: 0.5568 - accuracy: 0.7303
Epoch 7/20
858/858 [==============================] - 2s 3ms/step - loss: 0.5552 - accuracy: 0.7315
Epoch 8/20
858/858 [==============================] - 3s 3ms/step - loss: 0.5537 - accuracy: 0.7311
Epoch 9/20
858/858 [==============================] - 2s 3ms/step - loss: 0.5522 - accuracy: 0.7319
Epoch 10/20
858/858 [==============================] - 4s 4ms/step - loss: 0.5515 - accuracy: 0.7316

In [42]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


215/215 - 0s - loss: 0.5493 - accuracy: 0.7312 - 448ms/epoch - 2ms/step
Loss: 0.5492578744888306, Accuracy: 0.7311561703681946


In [43]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimisation.h5')
files.download('AlphabetSoupCharity_Optimisation.h5')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>